# LCEL: LangChain Expression Language
It composes chains of components.

A runnable protocol : 
        an allowable set of input types
        Required methods ( invoke, stream, batch)
        output type
      Also Modifying parameters at runtime (bind)
      


In [1]:
import os  
import openai 
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path("raj.env"))
openai.api_key=os.getenv("OPENAI_API_KEY")

In [2]:
#!pip install pydantic==1.10.0
#!pip install --upgrade langchain
#!pip install docarray
#!pip install tiktoken

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# A Simple Chain

The chain consists of a prompt | llm | outputparser.

It uses a Linux pip syntax to chain different componnet together with composition.  

In the next example we are going to do the following :
        1. First we write the prompt 
        2. Call the model for us it is openai
        3. We decide on how to handle the output. 
        
Depending  on the situation we can invoke / strream / batch process the command.

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "bears"})  ## the input is the input to the prompt template.


"Why don't bears wear shoes?\n\nBecause they have bear feet!"

# A Complex Chain with Vector Store

Here a vector store will be created that will be used as a retriver. 

This is a RAG style call to the openai. For the embedding we use openai embedding. 



In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [6]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [7]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [8]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [10]:

from langchain.schema.runnable import RunnableMap
inputs= RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) 
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


chain = inputs | prompt | model | output_parser

chain.invoke({"question": "where did harrison work?"})


'Harrison worked at Kensho.'

In [11]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

# Bind
Bind parameter to the invoke.


In the following we do the following

We call the openai. Next we write a function and bind (model = ChatOpenAI(temperature=0).bind(functions=functions) ) it . Then we modify the functions and again bind it. It is an example of how to add differnet parameters to the llm calls.

In [12]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions) ## This is the use of bind

In [14]:
runnable = prompt | model

In [15]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'weather_search'}})

In [16]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [17]:
model = model.bind(functions=functions)

In [18]:
runnable = prompt | model

In [19]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "team_name": "patriots"\n}', 'name': 'sports_search'}})

# Fallbacks

In [20]:
from langchain.llms import OpenAI
import json

The idea here is that the model will break as the model is an old one and does not return json output. 

In [21]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001"
)
simple_chain = simple_model | json.loads

In [22]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [23]:
simple_model.invoke(challenge)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-davinci-001` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [28]:
simple_chain.invoke(challenge)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-davinci-001` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [29]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [30]:
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [31]:
final_chain = simple_chain.with_fallbacks([chain])  
## the simple_chain is called and in the fallbacks all the runnable ([chain]) (here only one) go through.
## So it provide redundancy to the method final_chain

In [32]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

# Interface

invoke: one input call, synchrounous
batch :  multiple inputs
stream :   iterable 
(a) * : corresponding asynchrounous method

In [33]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [34]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

In [35]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 "Why don't frogs make good lawyers?\n\nBecause they always ribbit the wrong answer!"]

In [36]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



In [37]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

# Notes: 

The lesson for this notebook is to learn how to use LCEL. 

1. How to create a simple chain. A chain consists of consecutive tasks or action that can be called using the pipe syntyx used in Linux. 
2. For example : chain = prompt | model | output_parser is a simple chain here propmpt is the prompt to the llm. model is the llm of choice.  output_parser can be anyting such as json or str. 
3. Later a complex chain is used where a vectordb with openai embedding is used. 
4. Bind: This is to add or modify functionality of the model.
5. Fallbacks:  It provide redundancy to the model in case there is any error.
6. Interface: Diferent types of interfaces : invoke, stream, batach and their asynchrounous versions.